# Setup

TO DO:
- Different boat validation set
- Bring in listed external datasets
- Increase input image size
- Normalise the data
- Look at misclassifications for further ideas

- lrg_model.optimizer.lr=1e-5
- lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=6, validation_data=(conv_val_feat, val_labels))


In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import keras
import vgg16
import os
import numpy as np

from vgg16 import Vgg16
from inceptionV3 import IncepV3
from glob import glob
from keras.layers.core import Dense
from keras.preprocessing import image

Using TensorFlow backend.


In [3]:
#model = Vgg16()
model = IncepV3()

In [4]:
#path = 'data/'
path = 'data/sample_fish/'

# Create Validation Set

In [ ]:
for sub_path in os.listdir(path+'/train/'):
    g = glob(path+'train/'+sub_path+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(int(len(g)*0.1)):
        os.rename(shuf[i], path+'valid/' + sub_path + '/' + shuf[i].split('/')[-1])

# Fit a Model

In [5]:
bs=128
noe=6

In [6]:
batches = model.get_batches(path+'train', batch_size=bs)
valid_batches = model.get_batches(path+'valid', batch_size=bs*2, gen=image.ImageDataGenerator())

Found 11 images belonging to 8 classes.
Found 11 images belonging to 8 classes.


In [7]:
%autoreload
model.create(batches.nb_class)
model.finetune(batches, 0.001)

/Users/simonworgan/anaconda/lib/python3.5/site-packages/Keras-1.2.0-py3.5.egg/keras/applications/inception_v3.py:291: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image dimension ordering convention (`image_dim_ordering="th"`). For best performance, set `image_dim_ordering="tf"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [8]:
model.fit(batches, valid_batches, nb_epoch=noe)

Epoch 1/6


ValueError: Error when checking model input: expected input_1 to have shape (None, 3, 360, 640) but got array with shape (11, 3, 299, 299)

In [10]:
model.model.save_weights('incep_finetune1.h5')

# Write out Predictions

In [11]:
test_aug = 10

for i in range(test_aug):
    batches, preds = model.test(path+'test_stg1', batch_size = bs*2)
    if i == 0:
        predictions = preds
    else:
        predictions += preds

predictions /= test_aug

Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.


In [12]:
filenames = batches.filenames

with open("sample_sub.csv", "w") as f:
    f.write("image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n")
    for pred,fn in zip(predictions, filenames):
        pred = ",".join(map(lambda x: str(x), pred))
        f.write(fn+","+pred+"\n")

# Analyse Results

In [ ]:
model.model.load_weights('finetune1.h5')

In [ ]:
batches.filenames